# Key Value Format


In [49]:
import pandas as pd
import json
import requests
from tqdm import tqdm

# === CONFIGURATION ===
neptune_endpoint = "https://db-neptune-1.cluster-c8qttgkgfep5.us-east-1.neptune.amazonaws.com:8182/gremlin"
headers = {"Content-Type": "application/json"}

def send_gremlin(query):
    payload = {"gremlin": query}
    try:
        response = requests.post(neptune_endpoint, headers=headers, data=json.dumps(payload))
        if response.status_code != 200:
            print(f"❌ Query failed:\n{query}\nResponse:\n{response.text}")
        return response.status_code == 200
    except Exception as e:
        print(f"❌ Exception: {e}")
        return False

# === LOAD DATA ===
df = pd.read_excel("ISO_Main1.xlsx")
df.dropna(subset=['Company_Name', 'Attribute', 'Attribute_Value'], inplace=True)

# === CLEAN STRINGS ===
df['Company_Name'] = df['Company_Name'].astype(str).str.replace("'", "")
df['Attribute'] = df['Attribute'].astype(str).str.replace("'", "")
df['Attribute_Value'] = df['Attribute_Value'].astype(str).str.replace("'", "")

companies = set(df['Company_Name'])
attributes = set(df['Attribute'])
values = set(df['Attribute_Value'])

# === STEP 1: Create nodes ===
for company in tqdm(companies, desc="Creating Company Nodes"):
    q = f"g.V().has('Company', 'name', '{company}').fold().coalesce(unfold(), addV('Company').property('name', '{company}'))"
    send_gremlin(q)

for value in tqdm(values, desc="Creating AttributeValue Nodes"):
    q = f"g.V().has('AttributeValue', 'value', '{value}').fold().coalesce(unfold(), addV('AttributeValue').property('value', '{value}'))"
    send_gremlin(q)

for attr in tqdm(attributes, desc="Creating AttributeType Nodes"):
    q = f"g.V().has('AttributeType', 'type', '{attr}').fold().coalesce(unfold(), addV('AttributeType').property('type', '{attr}'))"
    send_gremlin(q)

# === STEP 2: Create edges ===
for _, row in tqdm(df.iterrows(), total=len(df), desc="Creating HAS_VALUE and HAS_TYPE Edges"):
    company = row['Company_Name']
    val = row['Attribute_Value']
    typ = row['Attribute']

    # Company → AttributeValue
    q1 = f"""
    g.V().has('Company', 'name', '{company}').as('c')
      .V().has('AttributeValue', 'value', '{val}')
      .coalesce(__.inE('HAS_VALUE').where(outV().as('c')), addE('HAS_VALUE').from('c'))
    """
    send_gremlin(q1)

    # AttributeValue → AttributeType
    q2 = f"""
    g.V().has('AttributeValue', 'value', '{val}').as('v')
      .V().has('AttributeType', 'type', '{typ}')
      .coalesce(__.inE('HAS_TYPE').where(outV().as('v')), addE('HAS_TYPE').from('v'))
    """
    send_gremlin(q2)

# === STEP 3: Create LINKED_COMPANY edges ===
grouped = df.groupby(['Attribute_Value', 'Attribute'])
for (val, typ), group in tqdm(grouped, desc="Creating LINKED_COMPANY Edges"):
    companies = group['Company_Name'].unique()
    if len(companies) > 1:
        for src in companies:
            for tgt in companies:
                if src != tgt:
                    q = f"""
                    g.V().has('AttributeType', 'type', '{typ}').as('t')
                      .V().has('Company', 'name', '{tgt}')
                      .coalesce(__.inE('LINKED_COMPANY').where(outV().as('t')), addE('LINKED_COMPANY').from('t').property('via', '{val}'))
                    """
                    send_gremlin(q)


Creating LINKED_COMPANY Edges: 100%|██████████| 46/46 [00:00<00:00, 220.33it/s]


In [48]:
send_gremlin("g.V().drop()")

True

# 3NF - Using Linkage1 & Company

In [42]:
import pandas as pd
import re
import requests
from tqdm import tqdm

# === CONFIGURATION ===
neptune_endpoint = "https://db-neptune-1.cluster-c8qttgkgfep5.us-east-1.neptune.amazonaws.com:8182/gremlin"
headers = {"Content-Type": "application/json"}

# === Helper Functions ===
def sanitize_edge(colname: str) -> str:
    label = colname.lower()
    label = re.sub(r'[^a-z0-9]', '_', label)
    label = re.sub(r'_+', '_', label).strip('_')
    return label

def send_gremlin(query: str):
    payload = {"gremlin": query}
    try:
        resp = requests.post(neptune_endpoint, headers=headers, json=payload)
        if resp.status_code != 200:
            print(f"❌ Query failed ({resp.status_code}): {resp.text}\nQuery: {query}")
        return resp.status_code == 200
    except Exception as e:
        print(f"❌ Exception: {e}")
        return False

# === Wipe the graph ===
print("🗑 Wiping the existing graph...")
send_gremlin("g.V().drop().iterate()")

# === Load Both Tables ===
print("📥 Loading ISO_Linkage1.xlsx...")
linkage_df = pd.read_excel("ISO_Linkage1.xlsx")
linkage_df.columns = linkage_df.columns.str.strip()
linkage_df = linkage_df.loc[:, ~linkage_df.columns.str.contains('^Unnamed')]

print("📥 Loading ISO_Company.xlsx...")
company_df = pd.read_excel("ISO_Company.xlsx")
company_df.columns = company_df.columns.str.strip()
company_df = company_df.loc[:, ~company_df.columns.str.contains('^Unnamed')]

# === Normalize Data ===
def clean(col):
    return col.astype(str).str.replace("'", "").str.strip()

linkage_df['Company Name'] = clean(linkage_df['Company Name'])
linkage_df['Linkage Method'] = clean(linkage_df['Linkage Method'])
linkage_df['Linkage Value Type'] = clean(linkage_df['Linkage Value Type'])
linkage_df['Linkage Value'] = clean(linkage_df['Linkage Value'])

company_df['Company'] = clean(company_df['Company'])
company_df['Payment method'] = clean(company_df['Payment method'])

# === Build Graph ===
print("🔨 Building Company, LinkageValue, and PaymentMethod nodes...")
value_to_companies = {}

# First handle Company nodes and Linkage values
for company, group in tqdm(linkage_df.groupby('Company Name'), desc="Companies"):
    send_gremlin(f"""
      g.V().has('Company','name','{company}').fold()
        .coalesce(unfold(), addV('Company').property('name','{company}'))
        .iterate()
    """)

    for _, row in group.iterrows():
        method = row['Linkage Method']
        val_type = row['Linkage Value Type']
        value = row['Linkage Value']

        if not (method and value):
            continue

        # 2) Upsert LinkageValue node
        send_gremlin(f"""
          g.V().has('LinkageValue','value','{value}').fold()
            .coalesce(unfold(), addV('LinkageValue').property('value','{value}'))
            .iterate()
        """)

        # 3) Edge: use special label if method is Chat
        if method.strip().lower() == "chat":
            edge = f"has_chat_{sanitize_edge(val_type)}"
        else:
            edge = f"has_{sanitize_edge(method)}"

        # Create the edge
        send_gremlin(f"""
          g.V().has('Company','name','{company}').as('c')
            .V().has('LinkageValue','value','{value}')
            .coalesce(
              __.inE('{edge}').where(outV().as('c')),
              addE('{edge}').from('c')
            ).iterate()
        """)

        value_to_companies.setdefault(value, set()).add(company)

# === Handle Payment Methods ===
print("💳 Adding Payment Methods...")
for idx, row in tqdm(company_df.iterrows(), total=company_df.shape[0], desc="Payment Methods"):
    company = row['Company']
    payment_method = row['Payment method']
    if pd.isna(payment_method) or payment_method == '':
        continue

    send_gremlin(f"""
      g.V().has('PaymentMethod','method','{payment_method}').fold()
        .coalesce(unfold(), addV('PaymentMethod').property('method','{payment_method}'))
        .iterate()
    """)

    send_gremlin(f"""
      g.V().has('Company','name','{company}').as('c')
        .V().has('PaymentMethod','method','{payment_method}')
        .coalesce(
          __.inE('uses_payment_method').where(outV().as('c')),
          addE('uses_payment_method').from('c')
        ).iterate()
    """)

# === Link Companies by Shared Linkage Values ===
print("🔗 Linking companies through shared values...")
for val, companies in tqdm(value_to_companies.items(), desc="Linking Companies"):
    if len(companies) < 2:
        continue
    companies = list(companies)
    for i in range(len(companies)):
        for j in range(i+1, len(companies)):
            a, b = companies[i], companies[j]
            q1 = f"""
            g.V().has('Company','name','{a}').as('x')
              .V().has('Company','name','{b}')
              .coalesce(
                __.inE('LinkedCompany').where(outV().as('x')).where(__.values('via').is('{val}')),
                addE('LinkedCompany').from('x').property('via','{val}')
              ).iterate()
            """
            send_gremlin(q1)
            q2 = f"""
            g.V().has('Company','name','{b}').as('x')
              .V().has('Company','name','{a}')
              .coalesce(
                __.inE('LinkedCompany').where(outV().as('x')).where(__.values('via').is('{val}')),
                addE('LinkedCompany').from('x').property('via','{val}')
              ).iterate()
            """
            send_gremlin(q2)

print("✅ Done building the updated dynamic graph!")


🗑 Wiping the existing graph...
📥 Loading ISO_Linkage1.xlsx...
📥 Loading ISO_Company.xlsx...
🔨 Building Company, LinkageValue, and PaymentMethod nodes...


Companies: 100%|██████████| 6/6 [00:01<00:00,  4.82it/s]


💳 Adding Payment Methods...


Payment Methods: 100%|██████████| 6/6 [00:00<00:00, 32.37it/s]


🔗 Linking companies through shared values...


Linking Companies: 100%|██████████| 21/21 [00:00<00:00, 204.55it/s]

✅ Done building the updated dynamic graph!


# Linkage method as center node

In [37]:
import pandas as pd
import re
import requests
from tqdm import tqdm

# === CONFIGURATION ===
neptune_endpoint = "https://db-neptune-1.cluster-c8qttgkgfep5.us-east-1.neptune.amazonaws.com:8182/gremlin"
headers = {"Content-Type": "application/json"}

# === Helper Functions ===
def sanitize(s: str) -> str:
    label = s.lower()
    label = re.sub(r'[^a-z0-9]', '_', label)
    label = re.sub(r'_+', '_', label).strip('_')
    return label

def send_gremlin(query: str):
    payload = {"gremlin": query}
    try:
        resp = requests.post(neptune_endpoint, headers=headers, json=payload)
        if resp.status_code != 200:
            print(f"❌ Query failed ({resp.status_code}): {resp.text}\nQuery: {query}")
        return resp.status_code == 200
    except Exception as e:
        print(f"❌ Exception: {e}")
        return False

# === Load Data ===
print("📥 Loading ISO_Linkage1.xlsx...")
df = pd.read_excel("ISO_Linkage1.xlsx")
df.columns = df.columns.str.strip()

# === Clean Columns ===
def clean(col):
    return col.astype(str).str.replace("'", "").str.strip()

df['Company Name'] = clean(df['Company Name'])
df['Linkage Method'] = clean(df['Linkage Method'])
df['Linkage Value Type'] = clean(df['Linkage Value Type'])
df['Linkage Value'] = clean(df['Linkage Value'])

# === Build Graph ===
print("🔨 Building graph from Linkage Method perspective...")
for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
    company = row['Company Name']
    method = row['Linkage Method']
    val_type = row['Linkage Value Type']
    value = row['Linkage Value']

    if not (company and method and value):
        continue

    # 1) Company Node
    send_gremlin(f"""
      g.V().has('Company','name','{company}').fold()
        .coalesce(unfold(), addV('Company').property('name','{company}'))
        .iterate()
    """)

    # 2) LinkageMethod Node
    send_gremlin(f"""
      g.V().has('LinkageMethod','method','{method}').fold()
        .coalesce(unfold(), addV('LinkageMethod').property('method','{method}'))
        .iterate()
    """)

    # 3) LinkageValue Node
    send_gremlin(f"""
      g.V().has('LinkageValue','value','{value}').fold()
        .coalesce(unfold(), addV('LinkageValue').property('value','{value}'))
        .iterate()
    """)

    if method.strip().lower() == "chat" and pd.notna(val_type):
        # 4) LinkageValueType Node
        send_gremlin(f"""
          g.V().has('LinkageValueType','type','{val_type}').fold()
            .coalesce(unfold(), addV('LinkageValueType').property('type','{val_type}'))
            .iterate()
        """)

        # LinkageMethod → has_type → LinkageValueType
        send_gremlin(f"""
          g.V().has('LinkageMethod','method','{method}').as('m')
            .V().has('LinkageValueType','type','{val_type}')
            .coalesce(
              __.inE('has_type').where(outV().as('m')),
              addE('has_type').from('m')
            ).iterate()
        """)

        # LinkageValueType → has_value → LinkageValue
        send_gremlin(f"""
          g.V().has('LinkageValueType','type','{val_type}').as('t')
            .V().has('LinkageValue','value','{value}')
            .coalesce(
              __.inE('has_value').where(outV().as('t')),
              addE('has_value').from('t')
            ).iterate()
        """)
    else:
        # LinkageMethod → has_value → LinkageValue
        send_gremlin(f"""
          g.V().has('LinkageMethod','method','{method}').as('m')
            .V().has('LinkageValue','value','{value}')
            .coalesce(
              __.inE('has_value').where(outV().as('m')),
              addE('has_value').from('m')
            ).iterate()
        """)

    # LinkageValue → used_by → Company
    send_gremlin(f"""
      g.V().has('LinkageValue','value','{value}').as('v')
        .V().has('Company','name','{company}')
        .coalesce(
          __.inE('used_by').where(outV().as('v')),
          addE('used_by').from('v')
        ).iterate()
    """)

print("✅ Graph created successfully from Linkage Method perspective!")


📥 Loading ISO_Linkage1.xlsx...
🔨 Building graph from Linkage Method perspective...


100%|██████████| 36/36 [00:03<00:00, 11.09it/s]

✅ Graph created successfully from Linkage Method perspective!


# Full Script for 3NF

In [ ]:
# Place this at the top of your script:
import pandas as pd
import re
import requests
from tqdm import tqdm

neptune_endpoint = "https://db-neptune-1.cluster-c8qttgkgfep5.us-east-1.neptune.amazonaws.com:8182/gremlin"
headers = {"Content-Type": "application/json"}

def sanitize(s: str) -> str:
    label = s.lower()
    label = re.sub(r'[^a-z0-9]', '_', label)
    return re.sub(r'_+', '_', label).strip('_')

def send_gremlin(query: str):
    payload = {"gremlin": query}
    try:
        resp = requests.post(neptune_endpoint, headers=headers, json=payload)
        if resp.status_code != 200:
            print(f"❌ {resp.status_code} - {resp.text}\nQuery: {query}")
        return resp.status_code == 200
    except Exception as e:
        print(f"❌ Exception: {e}")
        return False

# === Load Data ===
linkage_df = pd.read_excel("ISO_Linkage1.xlsx")
company_df = pd.read_excel("ISO_Company.xlsx")

# Clean
def clean(col): return col.astype(str).str.replace("'", "").str.strip()
linkage_df['Company Name'] = clean(linkage_df['Company Name'])
linkage_df['Linkage Method'] = clean(linkage_df['Linkage Method'])
linkage_df['Linkage Value Type'] = clean(linkage_df['Linkage Value Type'])
linkage_df['Linkage Value'] = clean(linkage_df['Linkage Value'])
company_df['Company'] = clean(company_df['Company'])
company_df['Payment method'] = clean(company_df['Payment method'])

# Optional: Uncomment to wipe graph
# print("🗑 Wiping the existing graph...")
# send_gremlin("g.V().drop().iterate()")

# === Build Graph ===
value_to_companies = {}

for _, row in tqdm(linkage_df.iterrows(), total=linkage_df.shape[0]):
    company = row['Company Name']
    method = row['Linkage Method']
    val_type = row['Linkage Value Type']
    value = row['Linkage Value']

    if not (company and method and value): continue

    send_gremlin(f"g.V().has('Company','name','{company}').fold().coalesce(unfold(), addV('Company').property('name','{company}')).iterate()")
    send_gremlin(f"g.V().has('LinkageMethod','method','{method}').fold().coalesce(unfold(), addV('LinkageMethod').property('method','{method}')).iterate()")
    send_gremlin(f"g.V().has('LinkageValue','value','{value}').fold().coalesce(unfold(), addV('LinkageValue').property('value','{value}')).iterate()")

    # Add Company → LinkageValue direct edge
    if method.lower() == "chat":
        edge = f"has_chat_{sanitize(val_type)}"
    else:
        edge = f"has_{sanitize(method)}"
    send_gremlin(f"""
        g.V().has('Company','name','{company}').as('c')
          .V().has('LinkageValue','value','{value}')
          .coalesce(__.inE('{edge}').where(outV().as('c')), addE('{edge}').from('c'))
          .iterate()
    """)

    value_to_companies.setdefault(value, set()).add(company)

    # Build linkage method → value chain
    if method.lower() == "chat" and pd.notna(val_type):
        send_gremlin(f"g.V().has('LinkageValueType','type','{val_type}').fold().coalesce(unfold(), addV('LinkageValueType').property('type','{val_type}')).iterate()")
        send_gremlin(f"""
            g.V().has('LinkageMethod','method','{method}').as('m')
              .V().has('LinkageValueType','type','{val_type}')
              .coalesce(__.inE('has_type').where(outV().as('m')), addE('has_type').from('m')).iterate()
        """)
        send_gremlin(f"""
            g.V().has('LinkageValueType','type','{val_type}').as('t')
              .V().has('LinkageValue','value','{value}')
              .coalesce(__.inE('has_value').where(outV().as('t')), addE('has_value').from('t')).iterate()
        """)
    else:
        send_gremlin(f"""
            g.V().has('LinkageMethod','method','{method}').as('m')
              .V().has('LinkageValue','value','{value}')
              .coalesce(__.inE('has_value').where(outV().as('m')), addE('has_value').from('m')).iterate()
        """)

    send_gremlin(f"""
        g.V().has('LinkageValue','value','{value}').as('v')
          .V().has('Company','name','{company}')
          .coalesce(__.inE('used_by').where(outV().as('v')), addE('used_by').from('v')).iterate()
    """)

# Add payment method nodes
for _, row in company_df.iterrows():
    company = row['Company']
    method = row['Payment method']
    if pd.isna(method) or method == '': continue
    send_gremlin(f"g.V().has('PaymentMethod','method','{method}').fold().coalesce(unfold(), addV('PaymentMethod').property('method','{method}')).iterate()")
    send_gremlin(f"""
        g.V().has('Company','name','{company}').as('c')
          .V().has('PaymentMethod','method','{method}')
          .coalesce(__.inE('uses_payment_method').where(outV().as('c')), addE('uses_payment_method').from('c')).iterate()
    """)

# Link companies via shared linkage values
for val, companies in value_to_companies.items():
    if len(companies) < 2: continue
    companies = list(companies)
    for i in range(len(companies)):
        for j in range(i+1, len(companies)):
            a, b = companies[i], companies[j]
            send_gremlin(f"""
                g.V().has('Company','name','{a}').as('x')
                  .V().has('Company','name','{b}')
                  .coalesce(__.inE('LinkedCompany').where(outV().as('x')).where(__.values('via').is('{val}')),
                            addE('LinkedCompany').from('x').property('via','{val}')).iterate()
            """)
            send_gremlin(f"""
                g.V().has('Company','name','{b}').as('x')
                  .V().has('Company','name','{a}')
                  .coalesce(__.inE('LinkedCompany').where(outV().as('x')).where(__.values('via').is('{val}')),
                            addE('LinkedCompany').from('x').property('via','{val}')).iterate()
            """)


  0%|          | 0/36 [00:00<?, ?it/s]

❌ Exception: HTTPSConnectionPool(host='db-neptune-1.cluster-c8qttgkgfep5.us-east-1.neptune.amazonaws.com', port=8182): Max retries exceeded with url: /gremlin (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f2731c345e0>: Failed to establish a new connection: [Errno 113] No route to host'))
❌ Exception: HTTPSConnectionPool(host='db-neptune-1.cluster-c8qttgkgfep5.us-east-1.neptune.amazonaws.com', port=8182): Max retries exceeded with url: /gremlin (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f2731ab8f70>: Failed to establish a new connection: [Errno 113] No route to host'))
❌ Exception: HTTPSConnectionPool(host='db-neptune-1.cluster-c8qttgkgfep5.us-east-1.neptune.amazonaws.com', port=8182): Max retries exceeded with url: /gremlin (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f2731c35e10>: Failed to establish a new connection: [Errno 113] No route to host'))
❌ Exception: HTTPSConnectionPoo

  3%|▎         | 1/36 [00:18<10:45, 18.43s/it]

❌ Exception: HTTPSConnectionPool(host='db-neptune-1.cluster-c8qttgkgfep5.us-east-1.neptune.amazonaws.com', port=8182): Max retries exceeded with url: /gremlin (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f2731c36c80>: Failed to establish a new connection: [Errno 113] No route to host'))
❌ Exception: HTTPSConnectionPool(host='db-neptune-1.cluster-c8qttgkgfep5.us-east-1.neptune.amazonaws.com', port=8182): Max retries exceeded with url: /gremlin (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f2731ab8e20>: Failed to establish a new connection: [Errno 113] No route to host'))
❌ Exception: HTTPSConnectionPool(host='db-neptune-1.cluster-c8qttgkgfep5.us-east-1.neptune.amazonaws.com', port=8182): Max retries exceeded with url: /gremlin (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f2731c36770>: Failed to establish a new connection: [Errno 113] No route to host'))
❌ Exception: HTTPSConnectionPoo

  6%|▌         | 2/36 [00:36<10:26, 18.43s/it]

❌ Exception: HTTPSConnectionPool(host='db-neptune-1.cluster-c8qttgkgfep5.us-east-1.neptune.amazonaws.com', port=8182): Max retries exceeded with url: /gremlin (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f2731ab8e20>: Failed to establish a new connection: [Errno 113] No route to host'))
❌ Exception: HTTPSConnectionPool(host='db-neptune-1.cluster-c8qttgkgfep5.us-east-1.neptune.amazonaws.com', port=8182): Max retries exceeded with url: /gremlin (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f2731ab9990>: Failed to establish a new connection: [Errno 113] No route to host'))
❌ Exception: HTTPSConnectionPool(host='db-neptune-1.cluster-c8qttgkgfep5.us-east-1.neptune.amazonaws.com', port=8182): Max retries exceeded with url: /gremlin (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f2731c37c70>: Failed to establish a new connection: [Errno 113] No route to host'))
❌ Exception: HTTPSConnectionPoo

  8%|▊         | 3/36 [01:01<11:40, 21.24s/it]

❌ Exception: HTTPSConnectionPool(host='db-neptune-1.cluster-c8qttgkgfep5.us-east-1.neptune.amazonaws.com', port=8182): Max retries exceeded with url: /gremlin (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f2731c36d40>: Failed to establish a new connection: [Errno 113] No route to host'))
❌ Exception: HTTPSConnectionPool(host='db-neptune-1.cluster-c8qttgkgfep5.us-east-1.neptune.amazonaws.com', port=8182): Max retries exceeded with url: /gremlin (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f2731c36980>: Failed to establish a new connection: [Errno 113] No route to host'))
❌ Exception: HTTPSConnectionPool(host='db-neptune-1.cluster-c8qttgkgfep5.us-east-1.neptune.amazonaws.com', port=8182): Max retries exceeded with url: /gremlin (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f2731c35fc0>: Failed to establish a new connection: [Errno 113] No route to host'))
❌ Exception: HTTPSConnectionPoo

 11%|█         | 4/36 [01:26<12:01, 22.56s/it]

❌ Exception: HTTPSConnectionPool(host='db-neptune-1.cluster-c8qttgkgfep5.us-east-1.neptune.amazonaws.com', port=8182): Max retries exceeded with url: /gremlin (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f2731c34c10>: Failed to establish a new connection: [Errno 113] No route to host'))
❌ Exception: HTTPSConnectionPool(host='db-neptune-1.cluster-c8qttgkgfep5.us-east-1.neptune.amazonaws.com', port=8182): Max retries exceeded with url: /gremlin (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f2731ab9c60>: Failed to establish a new connection: [Errno 113] No route to host'))
❌ Exception: HTTPSConnectionPool(host='db-neptune-1.cluster-c8qttgkgfep5.us-east-1.neptune.amazonaws.com', port=8182): Max retries exceeded with url: /gremlin (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f2731ab96f0>: Failed to establish a new connection: [Errno 113] No route to host'))
❌ Exception: HTTPSConnectionPoo

In [47]:
# Place this at the top of your script:
import pandas as pd
import re
import requests
from tqdm import tqdm

neptune_endpoint = "https://db-neptune-1.cluster-c8qttgkgfep5.us-east-1.neptune.amazonaws.com:8182/gremlin"
headers = {"Content-Type": "application/json"}

def sanitize(s: str) -> str:
    label = s.lower()
    label = re.sub(r'[^a-z0-9]', '_', label)
    return re.sub(r'_+', '_', label).strip('_')

def send_gremlin(query: str):
    payload = {"gremlin": query}
    try:
        resp = requests.post(neptune_endpoint, headers=headers, json=payload)
        if resp.status_code != 200:
            print(f"❌ {resp.status_code} - {resp.text}\nQuery: {query}")
        return resp.status_code == 200
    except Exception as e:
        print(f"❌ Exception: {e}")
        return False

# === Load Data ===
linkage_df = pd.read_excel("ISO_Linkage1.xlsx")
company_df = pd.read_excel("ISO_Company.xlsx")

# Clean
def clean(col): return col.astype(str).str.replace("'", "").str.strip()
linkage_df['Company Name'] = clean(linkage_df['Company Name'])
linkage_df['Linkage Method'] = clean(linkage_df['Linkage Method'])
linkage_df['Linkage Value Type'] = clean(linkage_df['Linkage Value Type'])
linkage_df['Linkage Value'] = clean(linkage_df['Linkage Value'])
company_df['Company'] = clean(company_df['Company'])
company_df['Payment method'] = clean(company_df['Payment method'])


# === Build Graph ===
value_to_companies = {}

for _, row in tqdm(linkage_df.iterrows(), total=linkage_df.shape[0]):
    company = row['Company Name']
    method = row['Linkage Method']
    val_type = row['Linkage Value Type']
    value = row['Linkage Value']

    if not (company and method and value): continue

    send_gremlin(f"g.V().has('Company','name','{company}').fold().coalesce(unfold(), addV('Company').property('name','{company}')).iterate()")
    send_gremlin(f"g.V().has('LinkageMethod','method','{method}').fold().coalesce(unfold(), addV('LinkageMethod').property('method','{method}')).iterate()")
    send_gremlin(f"g.V().has('LinkageValue','value','{value}').fold().coalesce(unfold(), addV('LinkageValue').property('value','{value}')).iterate()")

    # Add Company → LinkageValue direct edge
    if method.lower() == "chat":
        edge = f"has_chat_{sanitize(val_type)}"
    else:
        edge = f"has_{sanitize(method)}"
    send_gremlin(f"""
        g.V().has('Company','name','{company}').as('c')
          .V().has('LinkageValue','value','{value}')
          .coalesce(__.inE('{edge}').where(outV().as('c')), addE('{edge}').from('c'))
          .iterate()
    """)

    value_to_companies.setdefault(value, set()).add(company)

    # Build linkage method → value chain
    if method.lower() == "chat" and pd.notna(val_type):
        send_gremlin(f"g.V().has('LinkageValueType','type','{val_type}').fold().coalesce(unfold(), addV('LinkageValueType').property('type','{val_type}')).iterate()")
        send_gremlin(f"""
            g.V().has('LinkageMethod','method','{method}').as('m')
              .V().has('LinkageValueType','type','{val_type}')
              .coalesce(__.inE('has_type').where(outV().as('m')), addE('has_type').from('m')).iterate()
        """)
        send_gremlin(f"""
            g.V().has('LinkageValueType','type','{val_type}').as('t')
              .V().has('LinkageValue','value','{value}')
              .coalesce(__.inE('has_value').where(outV().as('t')), addE('has_value').from('t')).iterate()
        """)
    else:
        send_gremlin(f"""
            g.V().has('LinkageMethod','method','{method}').as('m')
              .V().has('LinkageValue','value','{value}')
              .coalesce(__.inE('has_value').where(outV().as('m')), addE('has_value').from('m')).iterate()
        """)



# Add payment method nodes
for _, row in company_df.iterrows():
    company = row['Company']
    method = row['Payment method']
    if pd.isna(method) or method == '': continue
    send_gremlin(f"g.V().has('PaymentMethod','method','{method}').fold().coalesce(unfold(), addV('PaymentMethod').property('method','{method}')).iterate()")
    send_gremlin(f"""
        g.V().has('Company','name','{company}').as('c')
          .V().has('PaymentMethod','method','{method}')
          .coalesce(__.inE('uses_payment_method').where(outV().as('c')), addE('uses_payment_method').from('c')).iterate()
    """)

# Link companies via shared linkage values
for val, companies in value_to_companies.items():
    if len(companies) < 2: continue
    companies = list(companies)
    for i in range(len(companies)):
        for j in range(i+1, len(companies)):
            a, b = companies[i], companies[j]
            send_gremlin(f"""
                g.V().has('Company','name','{a}').as('x')
                  .V().has('Company','name','{b}')
                  .coalesce(__.inE('LinkedCompany').where(outV().as('x')).where(__.values('via').is('{val}')),
                            addE('LinkedCompany').from('x').property('via','{val}')).iterate()
            """)
            send_gremlin(f"""
                g.V().has('Company','name','{b}').as('x')
                  .V().has('Company','name','{a}')
                  .coalesce(__.inE('LinkedCompany').where(outV().as('x')).where(__.values('via').is('{val}')),
                            addE('LinkedCompany').from('x').property('via','{val}')).iterate()
            """)


100%|██████████| 36/36 [00:09<00:00,  3.94it/s]
